Generate 112 trials (for now 1 only subject)

trials with shock after n seconds change physio signals
trials w\out shock no change in physio

In [1]:
# Load NeuroKit and other useful packages
import neurokit2 as nk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [2]:
TRIAL_LEN = 6
SHOCK_TIME = 4
AFTER_SHOCK = TRIAL_LEN-SHOCK_TIME
BPM_LOW = list(range(50,70))
BPM_HIGH = list(range(80,110))
DRIFT_RANGE= list(map(lambda x: x/1000.0, range(1, 10, 1)))

SEED = 1234
SAMPLING_RATE_HR = 500
SAMPLING_RATE_EDA = 500
THRESHOLD_BINARY = 2
assert SHOCK_TIME <= TRIAL_LEN

In [24]:
def generate_fake_eda_hr_shock(noise_hr=0.01,noise_eda=0.05,seed=None):
    random.seed(seed)
    ecg_low = nk.ecg_simulate(duration=SHOCK_TIME, noise=noise_hr, heart_rate=random.choice(BPM_LOW),sampling_rate=SAMPLING_RATE_HR)
    random.seed(seed)
    ecg_high = nk.ecg_simulate(duration=TRIAL_LEN-SHOCK_TIME, noise=noise_hr, heart_rate=random.choice(BPM_HIGH),sampling_rate=SAMPLING_RATE_HR)
    ecg=np.concatenate([ecg_low,ecg_high])

    while True:
        random.seed(seed)
        eda_test = nk.eda_simulate(duration=20, scr_number=2, noise=noise_eda,drift=random.choice(DRIFT_RANGE))
        signals, info = nk.eda_process(eda_test, sampling_rate=SAMPLING_RATE_EDA)

        if (info['SCR_Recovery'][0] is not None) and (info['SCR_Onsets'][1] is not None) and (not pd.isna(info['SCR_Recovery'][0])) and (not pd.isna(info['SCR_Onsets'][1])) :
            peak = int(info['SCR_Recovery'][0])
            peak_ = int(info['SCR_Onsets'][1])
            if peak_- peak > SAMPLING_RATE_EDA*TRIAL_LEN:
                break

    eda_low = eda_test[peak:peak_]
    eda_low = eda_low[-SHOCK_TIME*SAMPLING_RATE_EDA:]
    eda_high = eda_test[peak_:peak_+AFTER_SHOCK*SAMPLING_RATE_EDA]
    eda = np.concatenate([eda_low,eda_high])

    return ecg,eda

In [25]:
def generate_fake_eda_hr_noshock(noise_hr=0.01,noise_eda=0.05,seed=None):
    random.seed(seed)
    ecg = nk.ecg_simulate(duration=TRIAL_LEN, noise=noise_hr, heart_rate=random.choice(BPM_LOW),sampling_rate=SAMPLING_RATE_HR)

    while True:
        random.seed(seed)
        eda_test = nk.eda_simulate(duration=20, scr_number=2, noise=noise_eda,drift=random.choice(DRIFT_RANGE))
        signals, info = nk.eda_process(eda_test, sampling_rate=SAMPLING_RATE_EDA)

        if (info['SCR_Recovery'][0] is not None) and (info['SCR_Onsets'][1] is not None) and (not pd.isna(info['SCR_Recovery'][0])) and (not pd.isna(info['SCR_Onsets'][1])) :
            peak = int(info['SCR_Recovery'][0])
            peak_ = int(info['SCR_Onsets'][1])
            if (peak_- peak) > (SAMPLING_RATE_EDA*TRIAL_LEN):
                break

    eda_low = eda_test[peak:peak_]
    eda = eda_low[-TRIAL_LEN*SAMPLING_RATE_EDA:]

    return ecg,eda

In [5]:
ecg_high, eda_high = generate_fake_eda_hr_shock()
ecg_low,eda_low = generate_fake_eda_hr_noshock()

In [26]:
df = pd.read_csv('csv_shock.csv')
df.drop(columns=df.columns[0], axis=1, inplace=True)
df_subj = df['subject'].unique()

In [16]:
df[df['subject']==2]

,subject,trial,rating,shock
0,2,1,3.0,False
1,2,2,3.0,False
2,2,3,3.0,False
3,2,4,3.0,False
4,2,5,3.0,False
...,...,...,...,...
155,2,156,1.0,False
156,2,157,1.0,False
157,2,158,1.0,False
158,2,159,1.0,False


In [27]:
for subject in df_subj:

    df_eda = list()
    df_hr = list()
    #df_eda = [list(range(SAMPLING_RATE_EDA))]
    df_tmp = df[df['subject']==subject]

    for index,row in df_tmp.iterrows():

        if row['rating'] > THRESHOLD_BINARY:
            hr,eda = generate_fake_eda_hr_shock()
        else:
            hr,eda = generate_fake_eda_hr_noshock()

        #df_eda.append(eda)
        df_hr.append(hr)

    df_hr= pd.DataFrame(df_hr)
    df_hr.to_csv(r'data_fake/hr/'+str(subject)+'_hr.csv', index=False)
    #df_eda = pd.DataFrame(df_eda)
    #df_eda.to_csv(r'data_fake/eda/'+str(subject)+'_eda.csv', index=False)
    print('subject done')

subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
subject done
